# Lab Assignment 1

IE7300 - Statistical Learning for Engineerings

Name: Dat H. Tran

SID: 002925316

In [1]:
import numpy
import pandas
import matplotlib.pyplot as mlp

C:\Users\thund\AppData\Local\Temp\ipykernel_47108\3902267410.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [225]:
D1 = pandas.read_csv("housing.csv", header=None)

In [226]:
D2 = pandas.read_csv("yachtData.csv", header=None)

In [227]:
D3 = pandas.read_csv("concreteData.csv", header=None)

In [38]:
len(numpy.ones(D1.shape[0]))

1030

In [155]:
import enum

class LRMode(enum.Enum):
    CLOSED_FORM_SOLUTION = 0
    GRADIENT_DESCENT = 1,
    STOCHASTIC_GRADIENT_DESCENT = 2

class LinearRegression:
    def __init__(self, mode:LRMode = LRMode.CLOSED_FORM_SOLUTION, train_test_split = 0.2, regularization = 0, learning_rate = 0.01, n_iteration = 10000, tolerance = 0, minibatch = 32) -> None:
        # closed form solution by default
        self.mode = mode
        # Default split is 20% test
        self.train_test_split = train_test_split
        # Choose a positive value if regularization is desired
        self.regularization = max(regularization, 0)
        # Has no effect if the model is fit using closed form solution (mode = "closed_form")
        self.learning_rate = learning_rate
        # Should not exceed 50000. Has no effect if the model is fit using closed form solution (mode = "closed_form")
        self.n_iteration = min(n_iteration, 50000)
        # Choose a positive value to specify additional stopping condition for the training process. If the difference in root mean squared error between iteration is less than this value, the training stop. Has no effect if the model is fit using closed form solution (mode = "closed_form")
        self.tolerance = max(tolerance, 0)
        # Size of the minibatch to be used in SGD
        self.minibatch = max(minibatch, 1)

    def standardize(self, x:pandas.DataFrame) -> pandas.DataFrame:
        # store mean and std to be used as rescaling parameter for predict and testing
        self.nor_mean = x.mean()
        self.nor_std = x.std()
        return (x-self.nor_mean)/self.nor_std
    
    def RMSE(self, x:pandas.DataFrame):
        # split the target column
        test_x = x.iloc[:, 0:x.shape[1]-1]
        test_y = x.iloc[:, x.shape[1]-1]

        y_pred = numpy.dot(test_x, self.w)
        residuals = y_pred - test_y
        
        return numpy.sqrt(numpy.sum((residuals ** 2)) / test_x.shape[0])
    
    def SSE(self, x:pandas.DataFrame):
        # split the target column
        test_x = x.iloc[:, 0:x.shape[1]-1]
        test_y = x.iloc[:, x.shape[1]-1]

        y_pred = numpy.dot(test_x, self.w)
        residuals = y_pred - test_y
        
        return numpy.sum((residuals ** 2))


    def fit(self, d:pandas.DataFrame):
        # d is assumed to be a DataFrame object, contains all features and the target column as the last column
        # Store a shuffled copy of the input dataset
        self.D = d.sample(frac=1).copy()

        # Split data into train and test set
        t_size = int(self.D.shape[0] * self.train_test_split)
        self.test = self.D.iloc[0:t_size]
        self.train = self.D.iloc[t_size:]

        # standardize
        self.train = self.standardize(self.train)
        # standardize the test set using mean and std of the train set
        self.test = (self.test-self.nor_mean)/self.nor_std

        # add a bias column
        self.train.insert(0, "bias", numpy.ones(self.train.shape[0]), allow_duplicates=True)
        self.test.insert(0, "bias", numpy.ones(self.test.shape[0]), allow_duplicates=True)

        # split the target column
        train_x = self.train.iloc[:, 0:self.train.shape[1]-1]
        train_y = self.train.iloc[:, self.train.shape[1]-1]

        # store cost value of each iteration of the training
        self.costarray = []
        m = train_x.shape[0]

        # closed form solution
        if self.mode == LRMode.CLOSED_FORM_SOLUTION:
            # Solve system of linear equation instead of calculating matrix invert (essentially the same thing)
            # (X'*X + reg*I)*w = X'*Y
            self.w = numpy.linalg.solve(numpy.dot(train_x.T, train_x) + numpy.identity(train_x.shape[1])*self.regularization, numpy.dot(train_x.T, train_y))
            y_pred = numpy.dot(train_x, self.w)
            residuals = y_pred - train_y
            # cost is sum square error + regularization
            cost = numpy.sum((residuals ** 2)) + self.regularization * numpy.dot(self.w.T, self.w)
            self.costarray.append(cost)            
            return

        # gd
        if self.mode == LRMode.GRADIENT_DESCENT:
            # w is initiated as 0
            self.w = numpy.zeros(train_x.shape[1])        
            last_cost = 0
            for i in range(self.n_iteration):
                y_pred = numpy.dot(train_x, self.w)
                residuals = y_pred - train_y
                # gradient are individual for each coefficient, i.e. vector of gradients
                gradient = numpy.dot(train_x.T, residuals) + self.w*self.regularization*2
                # w = w' - learning_rate*gradient
                self.w -= self.learning_rate * gradient
                # cost is sum square error + regularization
                cost = numpy.sum((residuals ** 2)) + self.regularization * numpy.dot(self.w.T, self.w)
                # break loop if change in cost function is negligible
                if abs(last_cost - cost) < self.tolerance:
                    print (f'Training ended due to negligible improvement, iteration = {i}')
                    break
                last_cost = cost
                self.costarray.append(cost)
            return
        
        # sgd
        if self.mode == LRMode.STOCHASTIC_GRADIENT_DESCENT:
            # w is initiated as 0
            self.w = numpy.zeros(train_x.shape[1])        
            last_cost = 0

            # In each iteration, shuffle the training set, w is updated after every minibatch

            # Big iteration/epoch
            for i in range(self.n_iteration):
                shuffle_x = train_x.sample(frac=1)
                shuffle_y = train_y.reindex(shuffle_x.index)

                # Minibatch iterating
                index = 0
                index_u = 0
                br = False
                while (br==False):
                    if (index + self.minibatch > train_x.shape[0]):
                        br = True
                        index_u = train_x.shape[0]
                    else:
                        index_u = index + self.minibatch

                    batch_x = shuffle_x.iloc[index:index_u]
                    batch_y = shuffle_y.iloc[index:index_u]

                    y_pred = numpy.dot(batch_x, self.w)
                    residuals = y_pred - batch_y
                    # gradient are individual for each coefficient, i.e. vector of gradients
                    gradient = numpy.dot(batch_x.T, residuals) + self.w*self.regularization*2
                    # w = w' - learning_rate*gradient
                    self.w -= self.learning_rate * gradient

                    index = index_u

                y_pred = numpy.dot(shuffle_x, self.w)
                residuals = y_pred - shuffle_y         

                # cost is sum square error + regularization
                cost = numpy.sum((residuals ** 2)) + self.regularization * numpy.dot(self.w.T, self.w)
                # break loop if change in cost function is negligible
                if abs(last_cost - cost) < self.tolerance:
                    print (f'Training ended due to negligible improvement, iteration = {i}')
                    break
                last_cost = cost
                self.costarray.append(cost)            
            return

    
    # def predict(self, x):

    #     return numpy.dot(x, self.w)

    #def copy(self):
    #    return LinearRegression(self.learning_rate, self.n_iteration)
    #__copy__ = copy


(a) Housing: learning rate = 0.4 × 10−3, tolerance = 0.5 × 10−2

(b) Yacht: learning rate = 0.1 × 10−2, tolerance = 0.1 × 10−2

(c) Concrete: learning rate = 0.7 × 10−3, tolerance = 0.1 × 10−3

In [248]:
model = LinearRegression(mode=LRMode.CLOSED_FORM_SOLUTION, regularization=0.1, learning_rate=0.0004, n_iteration=1000, tolerance=0.005)
model.fit(D1)

In [249]:
print(f'Model weight:')
print(model.w)

if model.mode != LRMode.CLOSED_FORM_SOLUTION:
    mlp.plot(model.costarray)

print(f'Latest training cost: {model.costarray[len(model.costarray)-1]}')
print(f'Test set SSE: {model.SSE(model.test)}')
print(f'Test set RMSE: {model.RMSE(model.test)}')

Model weight:
[ 6.46588470e-16 -1.00716877e-01  1.00680462e-01  1.95283660e-02
  9.55889483e-02 -2.45788808e-01  2.70057627e-01  1.68946925e-02
 -3.39381138e-01  3.02400915e-01 -1.88789386e-01 -2.20532458e-01
  7.99813963e-02 -4.57737252e-01]
Latest training cost: 112.81512488954846
Test set SSE: 20.25136244012497
Test set RMSE: 0.4477817984366251


The performance of the model over each instance's test set is recorded as follow:

> Closed form solution

|    | Housing | Yacht | Concrete|
|----|------|------|-|
|SSE |20.25136244012497|16.323522202070187|68.68926588873637|
|RMSE|0.4477817984366251|0.5172994535316091|0.5774452686574172|


> Gradient Descent

|    | Housing | Yacht | Concrete|
|----|------|------|-|
|SSE |28.817485604767736|23.352912763440262|93.78442287119638|
|RMSE|0.5341550707904641|0.6187363214376814|0.6747326794332168|


> Stochastic gradient descent

|    | Housing | Yacht | Concrete|
|----|------|------|-|
|SSE |27.35117806513256|20.794298397333613|76.73146199768281|
|RMSE|0.5203880698278298|0.5838579773037333|0.6103137105058922|

